In [43]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SequentialFeatureSelector, f_classif, SelectKBest
from datetime import datetime
from sklearn.linear_model import LogisticRegression


In [44]:
df_train = pd.read_csv('../data/clean/df-train.csv')
df_test = pd.read_csv('../data/clean/df-test.csv')

" \ntransactions_df = pd.read_csv('../test_ml/transactions_df.csv')\nirrelevant_columns_df = pd.read_csv('../test_ml/irrelevant_columns_df.csv')\namount_outliers_df = pd.read_csv('../test_ml/amount_outliers_df.csv')\nage_loan_outliers_df = pd.read_csv('../test_ml/age_loan_outliers_df.csv')\n"

In [45]:
X_train = df_train.drop(columns='status', axis=1)
y_train = df_train[['status']]

X_test = df_test.drop(columns='status', axis=1)

In [46]:
oversample = True
feature_selection = False

pipeline = []
#model_instance = RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=200, n_jobs=-1)
model_instance = LogisticRegression(max_iter=5000, solver='saga', class_weight='balanced')
#rfe = SequentialFeatureSelector(model_instance, n_features_to_select="auto", tol=0.07)
rfe = SelectKBest(f_classif, k=10)

if oversample:
  pipeline.append(('sampling', SMOTE(n_jobs=-1)))

if feature_selection:
  pipeline.append(('rfe', rfe))

pipeline.append(("model", model_instance))

pipe = Pipeline(steps=pipeline)

In [47]:
pipe.fit(X_train, y_train)
y_result = pipe.predict_proba(X_test)[:, 0]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [9] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/imblearn/pipeline.py:272: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

In [48]:
result = pd.DataFrame({"Id": X_test["loan_id"], "Predicted": y_result})
result.drop_duplicates(inplace=True)
time = datetime.now().strftime('%d_%H-%M-%S')
result.to_csv(f"./results/result-{time}.csv", index=False)
with open(f"./results/result-{time}.txt", 'w') as f:
    f.write(f"Random Forest\n{'Feature Selection ' if feature_selection else ''}{'Oversample' if oversample else ''}\n{model_instance.get_params()}")